# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

Una **Feature** è contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi in cui deve essere implementata. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.
Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

Di seguito sono riportate le strutture utilizzate per modellare il problema.

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Tuple

@dataclass
class Binary:
    id: int
    services: List[int]
    working_on: int
    last_move: int
    last_work: int

    def can_move(self, day: int) -> bool:
        return max(self.last_move, self.last_work) <= day

    def can_work(self, day: int) -> bool:
        return self.last_move <= day
    


binaries: List[Binary] =  []
service_to_binaries: Dict[str, int] = {}
service_to_id: Dict[str, int] = {}

@dataclass
class Service:
    id: int
    name: str
    binary: int
    impl_features: List[str]

services: List[Service] = []
@dataclass
class Feature:
    idx: int
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    day_completed: int
    
    def is_finished(self) -> bool:
        for (service, completed) in self.dep_services:
            if not completed:
                return False
        return True

features: List[Feature] = []

## Strutture per lo scheduling
Ogni ingegnere è dotato di un array di operazioni da effettuare che tenere traccia delle sue attività. In particolare, la classe `Engineer` espone il metodo `add_op` che aggiunge l'operazione per un determinato giorno se e solo se è libero (grazie alla proprietà `avail`), altrimenti viene inserita un'operazione di `wait` della durata necessaria.

In [ ]:

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List[int]]]

    def add_op(self, op_name: str, day: int, duration: int, values: List[int]):
        assert(self.avail <= day)
        if self.avail < day:
            self.ops.append(("wait", [day - self.avail]))

        self.ops.append((op_name, values))
        self.avail = day + duration
        
engineers: List[Engineer] = []

### Lettura input da file
Come prima cosa vengono letti i dati input e si effettua il parsing della prima riga per avere la dimensione del problema.

In [3]:
fname = "data/e.in"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: dimensione del problema
L, G, S, B, F, N = [int(k) for k in lines[0].strip().split(" ")]


In [4]:
# modellazione del sistema
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set

score: int = 0
daily_schedule: List[Tuple[int, int]] = [] # giorno, binary

@dataclass
class Binary:
    id: int
    services: List[str]
    working_on: int
    locked: bool
binaries: List[Binary] =  [None] * B
service_to_binaries: Dict[str, int] = {}

@dataclass
class Service:
    name: str
    binary: int
    impl_features: List[str]

services: List[Service] = []

@dataclass
class Feature:
    idx: int
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    completed: bool
    binaries: Set[int]
    def populate_binaries(self):
        for (service, done) in self.dep_services:
            b = service_to_binaries[service]
            self.binaries.add(b)


features: List[Feature] = []

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List]]

    def add_op(self, op_name: str, day: str, duration: int, values: List[int]):
        if day > self.avail:
            self.ops.append(("wait", [day - self.avail]))

        affected_binaries = []
        if op_name == "move":
            service = services[values[0]]
            b1 = service_to_binaries[service.name]
            affected_binaries.append(b1, values[1])
        elif op_name == "impl":
            affected_binaries.append(values[1])
        elif op_name not in ["new", "wait"]:
            raise Exception(f"operation {op_name} not supported")

        # controllo se un binary coinvolto nell'operazione è bloccato
        # se si aggiungo una wait pari al massimo di quelli schedulati
        locked = [b for b in affected_binaries if binaries[b].locked]
        skip = []
        for l in locked:
            for ds in daily_schedule:
                if daily_schedule[ds] == l:
                    skip.append(ds)
        else:
            if len(skip) > 0:
                self.ops.append("wait", [day + skip])

        self.avail = day + duration
        self.ops.append((op_name, values))
engineers: List[Engineer] = []

### Parsing dell'input
In questa sezione, viene effettuato il parsing dei dati in ingresso creando:
* Lista dei binary: tutti i binary del sistema; per ogni binary si tiene traccia della list dei service installati e del numero di ingegneri che lavora al momento;
* Lista dei service; tutti i service del sistema. Per ogni service si tiene conto delle feature implementate al momento;
* Lista delle feature: tutte le feature del sistema. Ogni feature contiene una lista coppie (stringhe, booleani) in cui si tiene conto dei servizi da implementare e del loro stato (implementato si/no);

In [5]:
for i in range(G):
    engineers.append(Engineer(i, 0, []))

for (service, binary) in [line.strip().split() for line in lines[1:S+1]]:
    bid = int(binary)
    b = binaries[bid]
    if b is None:
        b = Binary(bid, [service], 0, False)
        binaries[bid] = b
    else:
        b.services.append(service)

    s = Service(service, bid, [])
    services.append(s)
    service_to_binaries[service] = bid
idx = 0
for i, feature in enumerate(lines[1+S:]):
    # leggo solo le righe pari
    if i % 2 ==0:
        feature, num_services, difficulty, users = feature.strip().split(" ")
        s = [[s, False] for s in lines[1+S+1+i].split()]
        f = Feature(idx, feature, int(difficulty), int(users), s, False, set())
        f.populate_binaries()
        features.append(f)
        idx += 1

print(engineers, binaries, service_to_binaries, features, services, sep="\n")

[Engineer(id=0, avail=0, ops=[]), Engineer(id=1, avail=0, ops=[]), Engineer(id=2, avail=0, ops=[]), Engineer(id=3, avail=0, ops=[]), Engineer(id=4, avail=0, ops=[]), Engineer(id=5, avail=0, ops=[]), Engineer(id=6, avail=0, ops=[]), Engineer(id=7, avail=0, ops=[]), Engineer(id=8, avail=0, ops=[]), Engineer(id=9, avail=0, ops=[]), Engineer(id=10, avail=0, ops=[]), Engineer(id=11, avail=0, ops=[]), Engineer(id=12, avail=0, ops=[]), Engineer(id=13, avail=0, ops=[]), Engineer(id=14, avail=0, ops=[]), Engineer(id=15, avail=0, ops=[]), Engineer(id=16, avail=0, ops=[]), Engineer(id=17, avail=0, ops=[]), Engineer(id=18, avail=0, ops=[]), Engineer(id=19, avail=0, ops=[]), Engineer(id=20, avail=0, ops=[]), Engineer(id=21, avail=0, ops=[]), Engineer(id=22, avail=0, ops=[]), Engineer(id=23, avail=0, ops=[]), Engineer(id=24, avail=0, ops=[]), Engineer(id=25, avail=0, ops=[]), Engineer(id=26, avail=0, ops=[]), Engineer(id=27, avail=0, ops=[]), Engineer(id=28, avail=0, ops=[]), Engineer(id=29, avail=0

## Attività
Di seguito sono illustrate le mosse che un ingegnere può effettuare.

Ciascuna mossa è caratterizzata da una durata. Per ogni mossa è definita una procedura per eseguirla sui dati di input. Ogni procedura ha la seguente firma:
* input:
    * ingegnere su cui schedulare l'attività;
    * giorno in cui effettuare l'attività;
    * feature, binary, service su cui effettuare l'attività;
* output:
    * durata dell'attività;

### Implementazione di una nuova feature
Su un *binary* $B_j$ può un ingegnere implementare tutti i servizi per una feature $F_i$. Questa mossa ha una durata $D_{Fi} + R_{Bj} + C_{Bj}$. Dove $D_{Fi}$ è data dalla complessità della **Feature**, $R_{Bj}$ è dato dal numero di servizi totali presenti nel *binary*, $C_{Bj}$ ingegneri che stanno già lavorando su $B_j$ nel giorno in cui viene *schedulata* l'attività. Inoltre, ogni ingegnere può implementare la stessa **Feature** su **binary** diversi. Infine, se più ingegneri sono *schedulati* per lavorare sullo stesso **binary** il primo della soluzione viene schedulato e conta per gli altri nel fattore $C_{Bj}$;


In [6]:
def implement_feature(e: Engineer, day: int, f: int, b: int) -> int:
    assert(b >= 0 and b < B)
    assert(day >=0 and day < L)
    assert(f >= 0 and f < F)
    global score
    feature = features[f]
    
    duration  = feature.difficulty + len(binaries[b].services) + binaries[b].working_on
    for service in feature.dep_services:
        s_name = service[0]
        done = service[1]
        if service_to_binaries[s_name] == b and not done:
            service[1] = True
    
    for (service, done) in feature.dep_services:
        if not done:
            break
    else:
        if not feature.completed:
            feature.completed = True
            # print("DEBUG: completed", feature, "multiply", 1 if  L-(day + duration) >= 0 else 0  )
            # print("DEBUG: remaining", len(features))
            score += feature.users * max(0, L-(day + duration)) 
    e.add_op("impl", day, duration, [f, b])
    daily_schedule.append((day + duration, b))
    binaries[b].working_on+=1
    
    return duration

### Migrazione di un service
un ingegnere può spostare un servizio $S_i$ da un *binary* $B_j$ a $B_k$, mantenendo le funzionalità implementate. Per fare questa mossa, un ingegnere impiega $\max (R_{bj}, R_{bk})$ giorni dove $R_{bj}, R_{bk}$ sono i servizi in esecuzione rispettivamente su $B_j$ a $B_k$. Quest'azione però è bloccante: nessun ingegnere può lavorare su $B_j$ e $B_k$ quando è in corso una migrazione; 

In [7]:
def move_service(e: Engineer, day: int, s: int, dst: int) -> int:
    assert(service_to_binaries[s] != dst)
    assert(day >= 0 and  day < L)
    assert(dst >= 0 and dst < B)
    service = services[s]
    src = service.binary
    duration  = max(len(binaries[src].services), len(binaries[dst].services))
    service_to_binaries[service.name] = dst
    binaries[src].services.remove(s)
    binaries[dst].services.append(s)
    binaries[src].locked = True
    binaries[dst].locked = True
    e.add_op("move", day, duration, [service.name, src, dst])
    return duration

### Nuovo binary
Un ingegnere può creare un nuovo binary senza nessun servizio al suo interno. Il costo per creare è un nuovo binary è fisso e vale $N$.


In [8]:
def new_binary(e: Engineer, day: int)-> int:
    assert(day >=0 and day < L)
    binaries.append(Binary(len(binaries)))
    e.add_op("new", day, N, [])
    return N

### Idle
Un ingegnere può essere in idle per un numero di giorni $1\leq W \leq L$

In [9]:
def wait(e: Engineer, day: int, duration: int) -> int:
    assert(day >=0 and day < L)
    e.add_op("wait", day, duration, [])
    return 1

## Euristica 
In una fase preliminare, l'algoritmo ordina le feature per numero di utenti soddisfatti. In questo approccio, si assegna un punteggio maggiore ai servizi che risiedono sulla stessa macchina.

In [10]:
features.sort(key=lambda x: (x.users-10*x.difficulty)/len(x.binaries), reverse=True)
# print(features[:10])

for day in range(L):
    while len(daily_schedule) > 0 and daily_schedule[-1][0] <= day:
        # migrazione
        if len(daily_schedule[-1]) == 3:
            b1 = daily_schedule[-1][1]
            b2 = daily_schedule[-1][2]
            binaries[b1].locked = False 
            binaries[b2].locked = False 
        elif len(daily_schedule[-1]) == 2:
            bid: int = daily_schedule[-1][1] 
            binaries[bid].working_on -=1
        else:
            raise Exception("unknown schedule event", daily_schedule[-1])
        daily_schedule.pop()
    
    current = 0
    for engineer in engineers:
        if engineer.avail <= day:
            completed = [k for k in features if k.completed]
            if len(completed) == len(features):
                break
            f: Feature = features[current]
            # print("doing", f)

            for bin in f.binaries:
                # next_b = service_to_binaries[dep[0]]
                # if next_b != b and not dep[1]:
                    implement_feature(engineer, day, current, bin)
                    if f.completed:
                        current += 1
                        break
           

num_eng: int = len([engineer for engineer in engineers if len(engineer.ops) > 0])
print(num_eng)
for engineer in engineers:
    if len(engineer.ops) == 0:
        continue
    print(engineer.id)
    for op in engineer.ops:
        print(op[0], end=" ")
        if op[0] == "impl":
            f = features[op[1][0]]
            print(f.name, op[1][1])
        elif op[0] == "move":
            print(op[1][0], op[1][2])
        elif op[0] == "wait":
            print(op[0], op[1][0])

print(score)
# print("features", features)
# print("impl features:", impl_features)
# sus = [f.users for f in impl_features]
# print(sum(sus))
# record 158.421.404

100
0
impl seacajee 1
1
impl sihaaddd 1
2
impl sebbaaif 1
3
impl sihbfedi 0
impl sihbfedi 1
4
impl sbbcchfb 0
5
impl sgfeaeej 0
impl sgfeaeej 1
6
impl sgfaddad 0
impl sgfaddad 1
7
impl sbcdadcj 1
8
impl shhejdab 0
9
impl sejcbdfe 0
10
impl secafejg 0
impl secafejg 1
11
impl sdaeabdf 1
12
impl sgfbafdh 0
impl sgfbafdh 1
13
impl sbcadbij 0
impl sbcadbij 1
14
impl sfgefcfh 0
impl sfgefcfh 1
15
impl shfdhehd 0
16
impl sbcbhiaa 0
impl sbcbhiaa 1
17
impl sdggidfg 0
impl sdggidfg 1
18
impl sibjccge 0
19
impl sibebcbb 0
impl sibebcbb 1
20
impl sgffhfgb 0
impl sgffhfgb 1
21
impl sedefdjc 1
22
impl sbbciefa 0
impl sbbciefa 1
23
impl siabdjfc 0
impl siabdjfc 1
24
impl sgdhijej 0
impl sgdhijej 1
25
impl sjbccicc 0
impl sjbccicc 1
26
impl seeecgii 0
impl seeecgii 1
27
impl scbheecj 0
impl scbheecj 1
28
impl shacfcbd 0
impl shacfcbd 1
29
impl sehbaiga 0
impl sehbaiga 1
30
impl sdcgjcbf 0
impl sdcgjcbf 1
31
impl sgeaicfb 0
impl sgeaicfb 1
32
impl scjajghj 0
33
impl sgcfbd 0
impl sgcfbd 1
34
impl sbib